<a href="https://colab.research.google.com/github/ElhamHosseini73/LearnNLP/blob/master/NaturalLanguageProcessingSpecialization/ClassificationAndVectorSpaces/Week2/Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


In [2]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

data = all_positive_tweets +all_negative_tweets

lable_positive = [1]* len(all_positive_tweets)
lable_negative = [0]* len(all_negative_tweets)

lables = lable_positive + lable_negative

In [3]:
#Train test split: 20% will be in the test set, and 80% in the training set.

len_pos_data_train = int(len(all_positive_tweets)*0.8)
len_neg_data_train = int(len(all_negative_tweets)*0.8)

x_train = all_positive_tweets[0:len_pos_data_train] + all_negative_tweets[0:len_neg_data_train]
x_test =  all_positive_tweets[len_pos_data_train:] + all_negative_tweets[len_neg_data_train:]

lable_train = lable_positive[0:len_pos_data_train]+lable_negative[0:len_pos_data_train]
lable_test =  lable_positive[len_pos_data_train:]+lable_negative[len_pos_data_train:]

In [4]:
#preprocessing twitter

import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import string
from nltk.stem import PorterStemmer

stopwords_english = stopwords.words('english')

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)


# Instantiate stemming class
stemmer = PorterStemmer()

def preprocessing_twitter(text):
  # remove old style retweet text "RT"
  text = re.sub(r'^RT[\s]+', '', text)

  # remove hyperlinks
  text = re.sub(r'https?://[^\s\n\r]+', '', text)

  # remove hashtags
  # only removing the hash # sign from the word
  text = re.sub(r'#', '', text)

  # tokenize tweets
  text_tokens = tokenizer.tokenize(text)

  text_clean = []

  for word in text_tokens: # Go through every word in your tokens list
    if (word not in stopwords_english and  # remove stopwords
        word not in string.punctuation):  # remove punctuation
        text_clean.append(word)

  text_stem = []

  for word in text_clean:
    stem_word = stemmer.stem(word)  # stemming word
    text_stem.append(stem_word)  # append to the list

  return text_stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# this function will build a dictionary where we can lookup how many times a word appears in the lists of positive or negative tweets.

def build_freqs(tweets, lables):

  #   Input: tweets: a list of tweets && lables: an m x 1 array with the sentiment label of each tweet (either 0 or 1)
  #   Output: freqs: a dictionary mapping each (word, lable) pair to its frequency

  word_freq = {}

  count_word_pos  = 0
  count_word_neg = 0

  for index , tweet in enumerate(tweets):
    process_tweet = preprocessing_twitter(tweet)

    for token in process_tweet:

      if (token,lables[index]) in word_freq:
        word_freq[token,lables[index]] = word_freq[token,lables[index]]+1
      else:
        word_freq[token,lables[index]] = 1

  return word_freq

In [6]:
#do for train data
word_freq = build_freqs(x_train,lable_train)

In [11]:
# it will build a dictionary where we can lookup the probability of the word condition the positive class and the same for the negative class

# count_word_pos and count_word_neg are the total freq number of positive and negative words for all documents respectively
count_word_pos = 0
count_word_neg = 0

for key in word_freq:
  if key[1] == 1:
    count_word_pos = count_word_pos + word_freq[key]
  if key[1] == 0:
    count_word_neg = count_word_neg + word_freq[key]

# count_word_corpus is the number of unique words in the entire set of documents, for all classes, whether positive or negative.
unique_strings = {key[0] for key in word_freq}
count_word_corpus = len(unique_strings)

word_probiblity ={}

for key in word_freq:
    if key[1] == 1:
        word_probiblity[key] = (word_freq.get(key)+1)/(count_word_pos+count_word_corpus)
        if (key[0],0) not in word_freq:
          word_probiblity[(key[0],0)] = 1/(count_word_pos+count_word_corpus)
    elif key[1] == 0:
        word_probiblity[key] = (word_freq.get(key)+1)/(count_word_neg+count_word_corpus)
        if (key[0],1) not in word_freq:
          word_probiblity[(key[0],1)] = 1/(count_word_neg+count_word_corpus)


In [30]:
# it will build a dictionary where key is word and value is likelihood for word
import numpy as np
word_likelihood = {}


for key in word_probiblity:

  if key[0] in word_likelihood:
    pass
  else:
    word_likelihood[key[0]] = np.log(float(word_probiblity[(key[0],1)])/float(word_probiblity[(key[0],0)]))

In [38]:
#calculate probability for each class is: the probability that the document is positive or negetive .

number_total_document = len(lable_train)
number_posetive_document = lable_train.count(1)
number_negetive_document = lable_train.count(0)

p_posetive = number_posetive_document/number_total_document
p_negetive = number_negetive_document/number_total_document

prior = np.log(p_posetive/p_negetive)

print(prior)



0.0
